In [ ]:
#Notebook description

#This notebook is being used to evaluate the techinical market conditions of a single asset and assess
#the appropriate strategy to take in order to maximize returns.

In [ ]:
#take all compuation functions and put them in a separate file

#simplify the date x axis on the percent drawdown chart

#default the zoom range to a comfortable range, and create a dropdown to select the time range for Volatility section

#properly label and annoate the garch models

#Remove the VIX charting, its redudnant now that we have the volatility models

In [ ]:
#Load libraries 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True
logger.propagate = False
import sys
sys.path.append(r"e:\Coding Projects\Investment Analysis")
# Load libraries
from Quantapp.Plotter import Plotter
from Quantapp.Computation import Computation, Helper, Models
from Quantapp.EconomicData import EconomicData

import numpy as np
import json
import os
import pandas as pd
import yfinance as yf
from statsmodels.tsa.stattools import coint
from IPython.display import display
from concurrent.futures import ThreadPoolExecutor
from plotly.subplots import make_subplots
from datetime import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

#shut down warnings
import warnings
warnings.filterwarnings("ignore")


qc = Computation()
qp = Plotter()
qe = EconomicData()
helper = Helper()
model = Models()


In [ ]:
#PARAMETERS (Adjust parmeters here)
# Define parameters for the analysis (Adjust these as needed)
interval = '1d'
period     = '20y'
risk_free_rate = 0.02 / 252  # Annualized risk-free rate divided by trading days
#should be a string or None
ticker_str = 'XLV' #ticker to analyze
primary_benchmark_str = 'SPY'# #primary benchmark to compare against
secondary_benchmark_str = None # #secondary benchmark to compare against
length_of_plots = 20 #number of years of data to plot (after computing the period, this will be adjusted to the closest available data)
trading_strategy = 'position'  # Options: 'position' or 'swing', # Determines the trading strategy to use for time frames

In [ ]:
# Load data
# Define timeframe lists for different strategies
position_time_frames = [21, 50, 200]  # For position trading (longer term)
swing_time_frames = [3, 9, 21]        # For swing trading (shorter term)

# Populate the time frame variables based on selected strategy
if trading_strategy == "position":
    time_frame_short = position_time_frames[0]
    time_frame_mid = position_time_frames[1]
    time_frame_long = position_time_frames[2]
else:  # swing trading
    time_frame_short = swing_time_frames[0]
    time_frame_mid = swing_time_frames[1]
    time_frame_long = swing_time_frames[2]
    

ticker = yf.Ticker(ticker_str).history(period=period, interval=interval)
vix = yf.Ticker('^VIX').history(period=period, interval=interval)

# Fetch and simplify benchmark data only if the strings are not None
if primary_benchmark_str is not None:
    primary_benchmark = yf.Ticker(primary_benchmark_str).history(period=period, interval=interval)
    primary_benchmark = helper.simplify_datetime_index(primary_benchmark)
    (primary_benchmark)
else:
    primary_benchmark = None

if secondary_benchmark_str is not None:
    secondary_benchmark = yf.Ticker(secondary_benchmark_str).history(period=period, interval=interval)
    secondary_benchmark = helper.simplify_datetime_index(secondary_benchmark)
else:
    secondary_benchmark = None

# Simplify ticker and vix
ticker = helper.simplify_datetime_index(ticker)
vix = helper.simplify_datetime_index(vix)

# Align ticker data with primary benchmark if available
if primary_benchmark is not None:
    ticker = ticker[ticker.index.isin(primary_benchmark.index)]

# Calculate ticker returns
ticker_monthly_returns = qc.calculate_returns(ticker, frequency='monthly')
ticker_weekly_returns = qc.calculate_returns(ticker, frequency='weekly')
ticker_daily_returns = qc.calculate_returns(ticker, frequency='daily')

# Calculate primary benchmark returns if it exists
if primary_benchmark is not None:
    primary_benchmark_monthly_returns = qc.calculate_returns(primary_benchmark, frequency='monthly')
    primary_benchmark_weekly_returns = qc.calculate_returns(primary_benchmark, frequency='weekly')
    primary_benchmark_daily_returns = qc.calculate_returns(primary_benchmark, frequency='daily')

# Calculate secondary benchmark returns if it exists
if secondary_benchmark is not None:
    secondary_benchmark_monthly_returns = qc.calculate_returns(secondary_benchmark, frequency='monthly')
    secondary_benchmark_weekly_returns = qc.calculate_returns(secondary_benchmark, frequency='weekly')
    secondary_benchmark_daily_returns = qc.calculate_returns(secondary_benchmark, frequency='daily')

# Calculate VIX returns
vix_monthly_returns = qc.calculate_returns(vix, frequency='monthly')
vix_weekly_returns = qc.calculate_returns(vix, frequency='weekly')
vix_daily_returns = qc.calculate_returns(vix, frequency='daily')


In [ ]:
#REGIME CHANGES: Candlestick with RSI and Bollinger Bands
qp.plot_candlestick(ticker, period='2y', bollinger_window=50, title="Candlestick with 50-Period Bollinger Bands")
qp.plot_candlestick(ticker, period='5y', bollinger_window=200, title="Candlestick with 200-Period Bollinger Bands ")

In [ ]:
#REGIME CHANGES: Percentage Drop from Highest Peak
n = int(252 / 2)
qp.plot_percentage_drop(ticker, n=n, title=f'{ticker_str} Percentage Drop from Highest Peak')

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# --- 1. Load your price series ---
# Replace this with your actual data loading code, make sure `ticker` DataFrame has 'Close' column
# For example:
# ticker = yf.download('SPY', start='2010-01-01', end='2025-07-12')
price_series = pd.Series(ticker['Close'].asfreq('D')).dropna()

# Define your window sizes as integers
window_sizes = list(range(3, 201))

# --- 2. Compute rolling Sharpe ratios and determine optimal window ---

def compute_optimal_momentum_window(price_series, windows):
    returns = price_series.pct_change()
    sharpe_df = pd.DataFrame(index=price_series.index)

    for w in windows:
        mean_return = returns.rolling(window=w).mean()
        std_return = returns.rolling(window=w).std()
        sharpe = mean_return / (std_return + 1e-8)  # prevent divide by zero
        sharpe_df[w] = sharpe

    # Drop rows where all Sharpe ratios are NaN (start of series)
    sharpe_df = sharpe_df.dropna(how='all')

    # Find the window with max Sharpe ratio at each date
    sharpe_df['Optimal_Window'] = sharpe_df.idxmax(axis=1).astype(float)
    return sharpe_df

sharpe_table = compute_optimal_momentum_window(price_series, window_sizes)

# --- 3. Plot optimal momentum window over time ---

fig_optimal = go.Figure()
fig_optimal.add_trace(go.Scatter(
    x=sharpe_table.index,
    y=sharpe_table['Optimal_Window'],
    mode='lines',
    name='Optimal Window Size'
))
fig_optimal.update_layout(
    title="Rolling Optimal Momentum Window",
    xaxis_title="Date",
    yaxis_title="Window Size (Days)",
    template="plotly_white",
    showlegend=False
)
fig_optimal.show()


import plotly.express as px

# Extract optimal window series, drop NaNs if any
optimal_windows = sharpe_table['Optimal_Window'].dropna()

# Convert to integers (window sizes are floats from idxmax)
optimal_windows_int = optimal_windows.astype(int)

# Plot histogram of optimal window sizes
fig_dist = px.histogram(
    optimal_windows_int,
    nbins=len(window_sizes),
    labels={'value': 'Optimal Window Size (Days)', 'count': 'Frequency'},
    title='Distribution of Optimal Sharpe Momentum Windows Over Time'
)

# plot vertical lines at 21, 50, and 200 days
fig_dist.add_vline(x=7, line_color='orange', line_dash='dash',
                    annotation_text="7 Days", annotation_position="top left")

fig_dist.add_vline(x=21, line_color='red', line_dash='dash',
                    annotation_text="21 Days", annotation_position="top left")
fig_dist.add_vline(x=50, line_color='blue', line_dash='dash',
                    annotation_text="50 Days", annotation_position="top left")
fig_dist.add_vline(x=200, line_color='green', line_dash='dash',
                    annotation_text="200 Days", annotation_position="top left")

fig_dist.update_layout(bargap=0.1)
fig_dist.show()

# --- 4. Calculate mean and median Sharpe ratios across all dates ---

# Exclude the 'Optimal_Window' column for stats
sharpe_only = sharpe_table.drop(columns='Optimal_Window')

mean_sharpe = sharpe_only.mean()
median_sharpe = sharpe_only.median()

fig_mean_median = go.Figure()
fig_mean_median.add_trace(go.Scatter(
    x=mean_sharpe.index,
    y=mean_sharpe.values,
    mode='lines+markers',
    name='Mean Sharpe Ratio'
))
fig_mean_median.add_trace(go.Scatter(
    x=median_sharpe.index,
    y=median_sharpe.values,
    mode='lines+markers',
    name='Median Sharpe Ratio'
))
fig_mean_median.update_layout(
    title="Mean and Median Sharpe Ratios Across All Dates by Momentum Window",
    xaxis_title="Momentum Window Size (Days)",
    yaxis_title="Sharpe Ratio",
    template="plotly_white"
)

# Plot vertical lines at 21, 50, and 200 days
fig_mean_median.add_vline(x=7, line_color='orange', line_dash='dash',
                    annotation_text="7 Days", annotation_position="top left")
fig_mean_median.add_vline(x=21, line_color='red', line_dash='dash',
                    annotation_text="21 Days", annotation_position="top left")
fig_mean_median.add_vline(x=50, line_color='blue', line_dash='dash',
                    annotation_text="50 Days", annotation_position="top left")
fig_mean_median.add_vline(x=200, line_color='green', line_dash='dash',
                    annotation_text="200 Days", annotation_position="top left")

fig_mean_median.show()

# --- 5. Calculate rolling volatility for each window ---

returns = price_series.pct_change()
volatility_df = pd.DataFrame(index=price_series.index)

for w in window_sizes:
    volatility_df[w] = returns.rolling(window=w).std()

volatility_df = volatility_df.reindex(sorted(volatility_df.columns), axis=1)

mean_volatility = volatility_df.mean()
median_volatility = volatility_df.median()

# --- 6. Plot mean and median volatility ---

fig_vol = go.Figure()
fig_vol.add_trace(go.Scatter(
    x=mean_volatility.index,
    y=mean_volatility.values,
    mode='lines+markers',
    name='Mean Volatility'
))
fig_vol.add_trace(go.Scatter(
    x=median_volatility.index,
    y=median_volatility.values,
    mode='lines+markers',
    name='Median Volatility'
))
fig_vol.update_layout(
    title="Mean and Median Volatility Across All Dates by Momentum Window",
    xaxis_title="Momentum Window Size (Days)",
    yaxis_title="Volatility (Rolling Std of Returns)",
    template="plotly_white"
)
#plot vertical line at 21, 50, and 200 days
fig_vol.add_vline(x=7, line_color='orange', line_dash='dash',
                    annotation_text="7 Days", annotation_position="top left")
fig_vol.add_vline(x=21, line_color='red', line_dash='dash',
                    annotation_text="21 Days", annotation_position="top left")
fig_vol.add_vline(x=50, line_color='blue', line_dash='dash',
                    annotation_text="50 Days", annotation_position="top left")
fig_vol.add_vline(x=200, line_color='green', line_dash='dash',
                    annotation_text="200 Days", annotation_position="top left")

fig_vol.show()

print("Mean volatility at selected windows:")
print(mean_volatility.loc[[5, 21, 50, 100, 150, 200]])
print("\nMedian volatility at selected windows:")
print(median_volatility.loc[[5, 21, 50, 100, 150, 200]])

# --- 7. 3D Surface plot of Sharpe ratios over last 10 years with vertical planes ---

# Filter to last 10 years of data
last_date = sharpe_only.index[-1]
ten_years_ago = last_date - pd.Timedelta(days=365*10)
sharpe_last_10y = sharpe_only.loc[ten_years_ago:last_date]

# Create integer y-axis for dates (since 3D plots need numeric axes)
date_labels = sharpe_last_10y.index.strftime('%Y-%m-%d')
y_vals = np.arange(len(sharpe_last_10y))

def create_vertical_plane(x_val, y_vals, z_min, z_max, opacity=0.3, color='red'):
    y_grid, z_grid = np.meshgrid(y_vals, np.linspace(z_min, z_max, 2))
    x_grid = np.full_like(y_grid, x_val)
    return go.Surface(
        x=x_grid,
        y=y_grid,
        z=z_grid,
        showscale=False,
        opacity=opacity,
        colorscale=[[0, color], [1, color]],
        hoverinfo='skip'
    )

surface_trace = go.Surface(
    z=sharpe_last_10y.values,
    x=sharpe_last_10y.columns,
    y=y_vals,
    colorscale='Viridis',
    colorbar=dict(title='Sharpe Ratio'),
    name='Sharpe Surface'
)

plane_21 = create_vertical_plane(21, y_vals, np.nanmin(sharpe_last_10y.values), np.nanmax(sharpe_last_10y.values), opacity=0.3, color='red')
plane_50 = create_vertical_plane(50, y_vals, np.nanmin(sharpe_last_10y.values), np.nanmax(sharpe_last_10y.values), opacity=0.3, color='blue')
plane_200 = create_vertical_plane(200, y_vals, np.nanmin(sharpe_last_10y.values), np.nanmax(sharpe_last_10y.values), opacity=0.3, color='green')

fig_3d = go.Figure(data=[surface_trace, plane_21, plane_50, plane_200])

fig_3d.update_layout(
    title='3D Surface of Sharpe Ratios by Window and Date (Last 10 Years) with Highlighted Windows',
    scene=dict(
        xaxis_title='Momentum Window Size (Days)',
        yaxis_title='Date',
        yaxis=dict(
            tickmode='array',
            tickvals=np.arange(0, len(date_labels), step=365),  # roughly yearly ticks
            ticktext=date_labels[::365]
        ),
        zaxis_title='Sharpe Ratio'
    ),
    height=900,
    template="plotly_white"
)

fig_3d.show()


In [ ]:
#Volatility: VIX FIX 

ticker_vix_fix = qc.vix_fix(ticker)

if primary_benchmark_str is not None:
    primary_benchmark_vix_fix = qc.vix_fix(primary_benchmark)
if secondary_benchmark_str is not None:
    secondary_benchmark_vix_fix = qc.vix_fix(secondary_benchmark)


qp.plot_series_with_stdev_bands(
    ticker_vix_fix,
    title='VIX Fix with Mean and Standard Deviations',
    stdev_values=[-0.5, 0.5, 1.5, 3]
)


In [ ]:
#Seasonality: Monthly, Weekly, and Daily Returns
#SEASONALITY: Monthly Seasonality
fig_ticker_Seasonality_Monthly = qp.plot_seasonality(ticker_monthly_returns, f'Monthly Seasonality: {ticker_str}', 'monthly')
fig_ticker_Seasonality_Monthly.show()

#SEASONALITY: Weekly Seasonality
fig_ticker_Seasonality_weekly = qp.plot_seasonality(ticker_weekly_returns, f'Weekly Seasonality: {ticker_str}', 'weekly')
fig_ticker_Seasonality_weekly.show()

#SEASONALITY: Daily Seasonality
fig_ticker_Seasonality_daily = qp.plot_seasonality(ticker_daily_returns, f'Daily Seasonality: {ticker_str}', 'daily')
fig_ticker_Seasonality_daily.show()

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def calculate_macd_log(price, short_window=12, long_window=26, signal_window=9):
    # Calculate log of close prices
    log_close = np.log(price['Close'])
    
    # Compute EMAs on log prices
    ema_short = log_close.ewm(span=short_window, adjust=True).mean()
    ema_long = log_close.ewm(span=long_window, adjust=True).mean()
    
    macd_line = ema_short - ema_long
    signal_line = macd_line.ewm(span=signal_window, adjust=True).mean()
    histogram = macd_line - signal_line
    
    return macd_line, signal_line, histogram

def calculate_zscore(series):
    return (series - series.mean()) / series.std()

def plot_macd_with_histogram_zscore(macd_line, signal_line, histogram, price_data, title="MACD (log prices) and Histogram Z-Score"):
    zscore_series = calculate_zscore(histogram.dropna())

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        row_heights=[0.4, 0.6], vertical_spacing=0.05,
                        subplot_titles=("MACD Line and Signal Line", "Histogram Z-Score"))

    # MACD line
    fig.add_trace(go.Scatter(
        x=macd_line.index, y=macd_line,
        mode='lines', name='MACD Line',
        line=dict(color='purple')
    ), row=1, col=1)

    # Signal line
    fig.add_trace(go.Scatter(
        x=signal_line.index, y=signal_line,
        mode='lines', name='Signal Line',
        line=dict(color='orange')
    ), row=1, col=1)

    # Histogram as bar chart
    fig.add_trace(go.Bar(
        x=histogram.index, y=histogram,
        name='Histogram',
        marker_color='gray'
    ), row=1, col=1)

    # Histogram z-score plot (bottom)
    fig.add_trace(go.Scatter(
        x=zscore_series.index, y=zscore_series,
        mode='lines', name='Histogram Z-Score',
        line=dict(color='blue')
    ), row=2, col=1)

    # Add horizontal lines and shaded regions for z-score
    levels = [0.5, 1, 1.5]
    for level in levels:
        fig.add_hline(y=level, line_color='red', line_dash='dash',
                      annotation_text=f"+{level} SD", annotation_position="top right", row=2, col=1)
        fig.add_hline(y=-level, line_color='green', line_dash='dash',
                      annotation_text=f"-{level} SD", annotation_position="bottom right", row=2, col=1)

    fig.add_hline(y=0, line_color='black', line_dash='dot', row=2, col=1)

    fig.add_shape(
        type="rect",
        xref="x", yref="y2",
        x0=zscore_series.index[0], x1=zscore_series.index[-1],
        y0=1, y1=1.5,
        fillcolor="rgba(255, 0, 0, 0.2)", line_width=0, layer="below",
        row=2, col=1
    )
    fig.add_shape(
        type="rect",
        xref="x", yref="y2",
        x0=zscore_series.index[0], x1=zscore_series.index[-1],
        y0=-1.5, y1=-1,
        fillcolor="rgba(0, 128, 0, 0.2)", line_width=0, layer="below",
        row=2, col=1
    )

    fig.update_layout(
        height=900,
        title=title,
        template="plotly_white",
        yaxis_title="MACD",
        yaxis2_title="Histogram Z-Score"
    )

    fig.show()

# Usage:
macd_12, signal_12, hist_12 = calculate_macd_log(ticker, 12, 26)
macd_21, signal_21, hist_21 = calculate_macd_log(ticker, 21, 50)
macd_50, signal_50, hist_50 = calculate_macd_log(ticker, 50, 200)
macd_200, signal_200, hist_200 = calculate_macd_log(ticker, 200, 400)

plot_macd_with_histogram_zscore(macd_12, signal_12, hist_12, ticker, title="Log-Price MACD and Histogram Z-Score (12, 26)")
plot_macd_with_histogram_zscore(macd_21, signal_21, hist_21, ticker, title="Log-Price MACD and Histogram Z-Score (21, 50)")
plot_macd_with_histogram_zscore(macd_50, signal_50, hist_50, ticker, title="Log-Price MACD and Histogram Z-Score (50, 200)")
plot_macd_with_histogram_zscore(macd_200, signal_200, hist_200, ticker, title="Log-Price MACD and Histogram Z-Score (200, 400)")




In [ ]:
#Calculations

sharpe_ticker_short = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sharpe', windows=[time_frame_short])[f'sharpe_ratio_{time_frame_short}']
sharpe_ticker_mid   = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sharpe', windows=[time_frame_mid])[f'sharpe_ratio_{time_frame_mid}']
sharpe_ticker_long  = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sharpe', windows=[time_frame_long])[f'sharpe_ratio_{time_frame_long}']

sortino_ticker_short = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sortino', windows=[time_frame_short])[f'sortino_ratio_{time_frame_short}']
sortino_ticker_mid   = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sortino', windows=[time_frame_mid])[f'sortino_ratio_{time_frame_mid}']
sortino_ticker_long  = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sortino', windows=[time_frame_long])[f'sortino_ratio_{time_frame_long}']

# calculate the spread between sharpe and sortino ratios of the ticker
sharpe_sortino_spread_short = sharpe_ticker_short - sortino_ticker_short
sharpe_sortino_spread_mid = sharpe_ticker_mid - sortino_ticker_mid
sharpe_sortino_spread_long = sharpe_ticker_long - sortino_ticker_long

if primary_benchmark is not None:
    spread_short_primary = primary_benchmark['Close'].pct_change(time_frame_short) - ticker['Close'].pct_change(time_frame_short)
    spread_mid_primary = primary_benchmark['Close'].pct_change(time_frame_mid) - ticker['Close'].pct_change(time_frame_mid)
    spread_long_primary = primary_benchmark['Close'].pct_change(time_frame_long) - ticker['Close'].pct_change(time_frame_long)
    
    sharpe_benchmark_short_primary = qc.calculate_risk_adjusted_returns(
        primary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_short]
    )[f'sharpe_ratio_{time_frame_short}']
    
    sharpe_benchmark_mid_primary = qc.calculate_risk_adjusted_returns(
        primary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_mid]
    )[f'sharpe_ratio_{time_frame_mid}']
    
    sharpe_benchmark_long_primary = qc.calculate_risk_adjusted_returns(
        primary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_long]
    )[f'sharpe_ratio_{time_frame_long}']
    
    sharpe_spread_short_primary = sharpe_benchmark_short_primary - sharpe_ticker_short
    sharpe_spread_mid_primary = sharpe_benchmark_mid_primary - sharpe_ticker_mid
    sharpe_spread_long_primary = sharpe_benchmark_long_primary - sharpe_ticker_long

# Calculate spreads and Sharpe ratios for the secondary benchmark if it exists
if secondary_benchmark is not None:
    spread_short_secondary = secondary_benchmark['Close'].pct_change(time_frame_short) - ticker['Close'].pct_change(time_frame_short)
    spread_mid_secondary = secondary_benchmark['Close'].pct_change(time_frame_mid) - ticker['Close'].pct_change(time_frame_mid)
    spread_long_secondary = secondary_benchmark['Close'].pct_change(time_frame_long) - ticker['Close'].pct_change(time_frame_long)
    
    sharpe_benchmark_short_secondary = qc.calculate_risk_adjusted_returns(
        secondary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_short]
    )[f'sharpe_ratio_{time_frame_short}']
    
    sharpe_benchmark_mid_secondary = qc.calculate_risk_adjusted_returns(
        secondary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_mid]
    )[f'sharpe_ratio_{time_frame_mid}']
    
    sharpe_benchmark_long_secondary = qc.calculate_risk_adjusted_returns(
        secondary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_long]
    )[f'sharpe_ratio_{time_frame_long}']
    
    sharpe_spread_short_secondary = sharpe_benchmark_short_secondary - sharpe_ticker_short
    sharpe_spread_mid_secondary = sharpe_benchmark_mid_secondary - sharpe_ticker_mid
    sharpe_spread_long_secondary = sharpe_benchmark_long_secondary - sharpe_ticker_long






In [ ]:
# Sharpe and Sortino Ratios with Spread Analysis
import plotly.graph_objects as go
import pandas as pd
import numpy as np

def plot_sharpe_sortino_and_spread(sharpe, sortino, time_frame, ticker_str, term_label, scale=None):
    # compute means
    mean_sharpe = sharpe.mean()
    mean_sortino = sortino.mean()

    # Plot Sharpe and Sortino ratios with mean lines
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=sharpe.index, y=sharpe, mode='lines',
        name=f'Sharpe Ratio ({time_frame}-day)', line=dict(color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=sortino.index, y=sortino, mode='lines',
        name=f'Sortino Ratio ({time_frame}-day)', line=dict(color='orange')
    ))
    fig.add_trace(go.Scatter(
        x=sharpe.index, y=pd.Series(mean_sharpe, index=sharpe.index),
        mode='lines', name=f'Mean Sharpe ({mean_sharpe:.3f})',
        line=dict(color='blue', dash='dash'), opacity=0.7
    ))
    fig.add_trace(go.Scatter(
        x=sortino.index, y=pd.Series(mean_sortino, index=sortino.index),
        mode='lines', name=f'Mean Sortino ({mean_sortino:.3f})',
        line=dict(color='orange', dash='dash'), opacity=0.7
    ))
    fig.update_layout(
        title=f'Sharpe and Sortino Ratios for {ticker_str} ({term_label})',
        xaxis_title='Date', yaxis_title='Ratio Value',
        template='plotly_white', legend=dict(x=0.01, y=0.99), height=500
    )
    fig.show()

    # Compute spread
    spread = (sharpe - sortino).dropna()

    if scale == 'z':
        # z-score scaling (uses running mean/std if preferred, replace with rolling if needed)
        spread_mean = spread.mean()
        spread_std = spread.std(ddof=0) if spread.std(ddof=0) != 0 else 1.0
        spread_z = (spread - spread_mean) / spread_std
        series_to_plot = spread_z
        y_title = 'Spread (z-score)'
    else:
        series_to_plot = spread
        y_title = 'Spread Value'

    # Plot spread (or z-score) and add reference lines
    fig_spread = go.Figure()
    fig_spread.add_trace(go.Scatter(
        x=series_to_plot.index, y=series_to_plot, mode='lines',
        name=f'Sharpe-Sortino Spread ({time_frame}-day)', line=dict(color='green')
    ))

    # zero / mean line
    fig_spread.add_shape(type="line",
        x0=series_to_plot.index.min(), x1=series_to_plot.index.max(),
        y0=0, y1=0, line=dict(color="black", width=1, dash="dash"),
        xref="x", yref="y"
    )

    # if scaled, draw ±1/±2 significance lines
    if scale == 'z':
        for lvl, color, dash in [(1, 'red', 'dot'), (2, 'purple', 'dot'), (-1, 'red', 'dot'), (-2, 'purple', 'dot')]:
            fig_spread.add_shape(type="line",
                x0=series_to_plot.index.min(), x1=series_to_plot.index.max(),
                y0=lvl, y1=lvl, line=dict(color=color, width=1, dash=dash),
                xref="x", yref="y"
            )
        # annotate current z-score mean at right edge
        cur_mean = series_to_plot.mean()
        fig_spread.add_trace(go.Scatter(
            x=[series_to_plot.index[-1]], y=[cur_mean],
            mode='markers+text', marker=dict(color='purple', size=6),
            text=[f'Mean z: {cur_mean:.2f}'], textposition='middle right', showlegend=False
        ))

    fig_spread.update_layout(
        title=f'Sharpe-Sortino Spread for {ticker_str} ({term_label})',
        xaxis_title='Date', yaxis_title=y_title,
        template='plotly_white', legend=dict(x=0.01, y=0.99), height=500
    )
    fig_spread.show()

# Short Term
plot_sharpe_sortino_and_spread(
    sharpe_ticker_short, sortino_ticker_short, time_frame_short, ticker_str
    , term_label='Short Term', scale='z'
    
)
# Mid Term
plot_sharpe_sortino_and_spread(
    sharpe_ticker_mid, sortino_ticker_mid, time_frame_mid, ticker_str
    , term_label='Mid Term', scale='z'
    
)
# Long Term
plot_sharpe_sortino_and_spread(
    sharpe_ticker_long, sortino_ticker_long, time_frame_long, ticker_str
    , term_label='Long Term', scale='z'
)

# ...existing code...

In [ ]:
#Benchmark Comparisons: Risk-Adjusted Returns and Return Plots

# Combine all risk-adjusted return plots into a single loop for short, mid, and long term

# Prepare benchmark lists for each term
benchmarks_short = []
if primary_benchmark is not None:
    benchmarks_short.append((primary_benchmark, primary_benchmark_str, sharpe_spread_short_primary, time_frame_short, length_of_plots))
if secondary_benchmark is not None:
    benchmarks_short.append((secondary_benchmark, secondary_benchmark_str, sharpe_spread_short_secondary, time_frame_short, length_of_plots))

benchmarks_mid = []
if primary_benchmark is not None:
    benchmarks_mid.append((primary_benchmark, primary_benchmark_str, sharpe_spread_mid_primary, time_frame_mid, length_of_plots))
if secondary_benchmark is not None:
    benchmarks_mid.append((secondary_benchmark, secondary_benchmark_str, sharpe_spread_mid_secondary, time_frame_mid, length_of_plots))

benchmarks_long = []
if primary_benchmark is not None:
    benchmarks_long.append((primary_benchmark, primary_benchmark_str, sharpe_spread_long_primary, time_frame_long, length_of_plots))
if secondary_benchmark is not None:
    benchmarks_long.append((secondary_benchmark, secondary_benchmark_str, sharpe_spread_long_secondary, time_frame_long, length_of_plots))

# Combine all benchmarks into one list with a label for the term
all_benchmarks = (
    [("Short Term",) + b for b in benchmarks_short] +
    [("Mid Term",) + b for b in benchmarks_mid] +
    [("Long Term",) + b for b in benchmarks_long]
)



for term, benchmark, benchmark_str, sharpe_spread, time_frame, default_years in all_benchmarks:
    qp.create_spread_plot(
        sharpe_spread,
        title=f'{time_frame} Day: Benchmark ({benchmark_str}) minus {ticker_str} (Sharpe) [{term}]',
        default_years=default_years
    ).show()

    qp.plot_returns(
        ticker['Close'],
        benchmark_series=benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame} days [{term}]',
        window=time_frame,
        default_years=default_years
    ).show()
# ...existing code...